In [7]:
import numpy as np
from typing import List, Tuple
from IPython.display import display, HTML
import verovio
import pandas as pd


A4_FREQUENCY = 440

def convert_frequencies(df: pd.DataFrame) -> Tuple[List[str], List[int], List[str]]:
    frequencies = df['Frequency (Hz)'].tolist()
    def acoustical_pitch_name(midi_num: int) -> str:
        pitch_names = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
        pitch_idx = midi_num % 12
        pitch_name = pitch_names[pitch_idx]
        octave = (midi_num - 12) // 12
        return f"{pitch_name}{octave}"

    def midi_value(log_freq_ratio: float) -> int:
        return int(round(12 * log_freq_ratio + 69))

    def midi_cent_value(log_freq_ratio: float) -> int:
        return int(round(69 * 100 + 1200 * log_freq_ratio))

    log_freq_ratios = [np.log2(freq / A4_FREQUENCY) for freq in frequencies]
    midi_values = [midi_value(log_freq_ratio) for log_freq_ratio in log_freq_ratios]
    midi_cent_values = [midi_cent_value(log_freq_ratio) for log_freq_ratio in log_freq_ratios]
    pitch_names = [acoustical_pitch_name(midi_value) for midi_value in midi_values]
    midi_cent_deviations = [((midi_cent_value - midi_value * 100) + 50) % 100 - 50 for midi_value, midi_cent_value in zip(midi_values, midi_cent_values)]
    midi_cent_deviations = [f'+{x}' if x >= 0 else f'{x}' for x in midi_cent_deviations]

    return pitch_names, midi_values, midi_cent_deviations

df = pd.read_csv('conga3.tsv', sep='\t')

pitch_names, midi_values, midi_cent_deviations = convert_frequencies(df)

print(midi_values)
print(midi_cent_deviations)
print(pitch_names)

def pitch_name_to_step_octave_alter(pitch_name: str) -> Tuple[str, int, int]:
    step = pitch_name[0]
    alter = 1 if '#' in pitch_name else -1 if 'b' in pitch_name else 0
    octave = int(pitch_name[-1])
    return step, octave, alter

def pitch_name_and_cent_deviation_to_step_octave_alter(pitch_name: str, cent_deviation: str) -> Tuple[str, int, str]:
    step = pitch_name[0]
    octave = int(pitch_name[-1])
    deviation = int(cent_deviation)
    alter = 'n'
    if '#' in pitch_name:
        alter = 's'
        if deviation >= 25:
            alter = '3qs'
        elif deviation > 0:
            alter = '1qs'
    elif 'b' in pitch_name:
        alter = 'f'
        if deviation <= -25:
            alter = '3qf'
        elif deviation < 0:
            alter = '1qf'
    else:
        if deviation >= 25:
            alter = '1qs'
        elif deviation > 0:
            alter = '1qs'
        elif deviation <= -25:
            alter = '1qf'
        elif deviation < 0:
            alter = '1qf'
    return step, octave, alter

notes_and_alters = [pitch_name_and_cent_deviation_to_step_octave_alter(pitch_name, cent_deviation) for pitch_name, cent_deviation in zip(pitch_names, midi_cent_deviations)]

print(notes_and_alters)

def create_mei_string(notes_and_alters: List[Tuple[str, int, str]], midi_values: List[int], staff: str) -> str:
    staff_defs = ''
    if 'G' in staff:
        staff_defs += '<staffDef n="1" lines="5" clef.shape="G" clef.line="2"/>\n'
    if 'F' in staff:
        staff_defs += '<staffDef n="2" lines="5" clef.shape="F" clef.line="4"/>\n'
    
    mei = f'''
    <?xml version="1.0" encoding="UTF-8"?>
    <mei xmlns="http://www.music-encoding.org/ns/mei" meiversion="4.0.1">
        <music>
            <body>
                <mdiv>
                    <score>
                        <scoreDef keysig="0" mode="major">
                            <staffGrp n="1">
                                {staff_defs}
                            </staffGrp>
                        </scoreDef>
                        <section>
    '''
    for i, ((step, octave, alter), midi_value) in enumerate(zip(notes_and_alters, midi_values)):
        mei += f'<measure n="{i+1}" right="invis">\n'
        if 'G' in staff:
            mei += '<staff n="1">\n<layer n="1">\n'
            if octave >= 4 or (octave == 3 and step == 'B'):
                mei += f'''
                                        <note dur="4" oct="{octave}" pname="{step.lower()}" pnum="{midi_value}" stem.dir="down" accid="{alter}" stem.visible="false"/>
                '''
            mei += '</layer>\n</staff>\n'
        if 'F' in staff:
            mei += '<staff n="2">\n<layer n="1">\n'
            if octave < 4 or (octave == 4 and step == 'C' and alter in ['f', '1qf', '3qf']):
                mei += f'''
                                        <note dur="4" oct="{octave}" pname="{step.lower()}" pnum="{midi_value}" stem.dir="down" accid="{alter}" stem.visible="false"/>
                '''
            mei += '</layer>\n</staff>\n'
        mei += '</measure>\n'
    mei += '''
                        </section>
                    </score>
                </mdiv>
            </body>
        </music>
    </mei>
    '''
    return mei


mei = create_mei_string(notes_and_alters, midi_values.copy(), staff='GF')

frequencies = df['Frequency (Hz)'].tolist()

# create a DataFrame
df_para = pd.DataFrame({
    'Frequency (Hz)': frequencies,
    'MIDI Value': midi_values,
    'MIDI Cent Deviation': midi_cent_deviations,
    'Pitch Name': pitch_names,
    'Note': [x[0] for x in notes_and_alters],
    'Octave': [x[1] for x in notes_and_alters],
    'Alter': [x[2] for x in notes_and_alters],
})

display(df_para)

# Create a VerovioToolkit object
vrvToolkit = verovio.toolkit()

# Set the rendering options
options = {
    "inputFormat": "mei",
    "scale": 50,
    "adjustPageHeight": True,
    "noHeader": True,
    "border": 0,
    "pageHeight": 1300,
    "pageWidth": 1800,
    "staffLineWidth": 1.5,
    "systemDividerLineWidth": 1.5
}

# Render the music notation and get the HTML code
htmlCode = vrvToolkit.renderData(mei, options=options)

# Display the HTML code in the notebook
display(HTML(htmlCode))


[24, 38, 47, 52, 54, 54, 58, 61, 62, 64, 64, 67, 67, 68, 69, 69, 70, 71, 71, 72, 72, 72, 73, 73, 74, 78, 79, 83, 87]
['-39', '+41', '+44', '-9', '-31', '+41', '+45', '+36', '+41', '-44', '-50', '+7', '+28', '-6', '-12', '+32', '-8', '-32', '+12', '-49', '-16', '+33', '-42', '-23', '+34', '-44', '-16', '+13', '-47']
['C1', 'D2', 'B2', 'E3', 'F#3', 'F#3', 'A#3', 'C#4', 'D4', 'E4', 'E4', 'G4', 'G4', 'G#4', 'A4', 'A4', 'A#4', 'B4', 'B4', 'C5', 'C5', 'C5', 'C#5', 'C#5', 'D5', 'F#5', 'G5', 'B5', 'D#6']
[('C', 1, '1qf'), ('D', 2, '1qs'), ('B', 2, '1qs'), ('E', 3, '1qf'), ('F', 3, 's'), ('F', 3, '3qs'), ('A', 3, '3qs'), ('C', 4, '3qs'), ('D', 4, '1qs'), ('E', 4, '1qf'), ('E', 4, '1qf'), ('G', 4, '1qs'), ('G', 4, '1qs'), ('G', 4, 's'), ('A', 4, '1qf'), ('A', 4, '1qs'), ('A', 4, 's'), ('B', 4, '1qf'), ('B', 4, '1qs'), ('C', 5, '1qf'), ('C', 5, '1qf'), ('C', 5, '1qs'), ('C', 5, 's'), ('C', 5, 's'), ('D', 5, '1qs'), ('F', 5, 's'), ('G', 5, '1qf'), ('B', 5, '1qs'), ('D', 6, 's')]


,Frequency (Hz),MIDI Value,MIDI Cent Deviation,Pitch Name,Note,Octave,Alter
0,31.966817,24,-39,C1,C,1,1qf
1,75.181218,38,+41,D2,D,2,1qs
2,126.683312,47,+44,B2,B,2,1qs
3,163.977932,52,-9,E3,E,3,1qf
4,181.737274,54,-31,F#3,F,3,s
5,189.432990,54,+41,F#3,F,3,3qs
6,239.159149,58,+45,A#3,A,3,3qs
7,282.965528,61,+36,C#4,C,4,3qs
8,300.724871,62,+41,D4,D,4,1qs
9,321.444104,64,-44,E4,E,4,1qf
